In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from tqdm import *
import pandas as pd

import time
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 셀레니움 진행 중 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re 
import random


서울시 강남구 일반음식점 인허가 정보 데이터셋

https://data.seoul.go.kr/dataList/OA-18674/S/1/datasetView.do

In [78]:
df = pd.read_csv('./data/서울시 강남구 일반음식점 인허가 정보.csv', encoding='cp949',encoding_errors='ignore')
df.head()


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3220000,3220000-101-1007-00001,19970205,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,0.00,NaN,NaN,NaN
1,3220000,3220000-101-1899-05119,19960829,NaN,3,폐업,2,폐업,19991014,NaN,...,0.0,0.0,NaN,0.0,0.0,N,23.29,NaN,NaN,NaN
2,3220000,3220000-101-1899-14279,19960201,NaN,3,폐업,2,폐업,19991005,NaN,...,0.0,0.0,NaN,0.0,0.0,N,94.38,NaN,NaN,NaN
3,3220000,3220000-101-1899-15993,19930614,NaN,3,폐업,2,폐업,19991004,NaN,...,0.0,0.0,NaN,0.0,0.0,N,73.43,NaN,NaN,NaN
4,3220000,3220000-101-1899-16683,19921002,NaN,3,폐업,2,폐업,19991009,NaN,...,0.0,0.0,NaN,0.0,0.0,N,212.94,NaN,NaN,NaN


In [79]:
df.isna().sum()

개방자치단체코드         0
관리번호             0
인허가일자            0
인허가취소일자      47193
영업상태코드           0
영업상태명            0
상세영업상태코드         0
상세영업상태명          0
폐업일자         12530
휴업시작일자       47193
휴업종료일자       47193
재개업일자        47193
전화번호         18065
소재지면적          361
소재지우편번호         19
지번주소            19
도로명주소        20553
도로명우편번호      20843
사업장명             0
최종수정일자           0
데이터갱신구분          0
데이터갱신일자          0
업태구분명            1
좌표정보(X)       1278
좌표정보(Y)       1278
위생업태명         7401
남성종사자수       21800
여성종사자수       21796
영업장주변구분명     22511
등급구분명        22729
급수시설구분명      19279
총인원          45590
본사종업원수       45580
공장사무직종업원수    45580
공장판매직종업원수    45580
공장생산직종업원수    45580
건물소유구분명      47192
보증액          45580
월세액          45580
다중이용업소여부      7401
시설총규모         7401
전통업소지정번호     47178
전통업소주된음식     47189
홈페이지         47193
dtype: int64

In [82]:
unique_values = df['영업상태명'].unique()
unique_values


array(['영업/정상', '폐업'], dtype=object)

In [83]:
df.drop(df[df['영업상태명'] == '폐업'].index, inplace=True)
df

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3220000,3220000-101-1007-00001,19970205,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,0.00,NaN,NaN,NaN
7,3220000,3220000-101-1899-20451,19991005,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,950.66,NaN,NaN,NaN
8,3220000,3220000-101-1899-20453,19991005,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,473.24,NaN,NaN,NaN
10,3220000,3220000-101-1899-20463,19991006,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,282.59,NaN,NaN,NaN
15,3220000,3220000-101-1899-20500,19991012,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,1157.90,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47186,3220000,3220000-101-2024-00031,2024-01-09,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47187,3220000,3220000-101-2024-00032,2024-01-09,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47188,3220000,3220000-101-2024-00033,2024-01-09,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47189,3220000,3220000-101-2024-00034,2024-01-10,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# Test
df_top10 = df.head(10)
df_top10


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3220000,3220000-101-1007-00001,19970205,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,0.00,NaN,NaN,NaN
7,3220000,3220000-101-1899-20451,19991005,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,950.66,NaN,NaN,NaN
8,3220000,3220000-101-1899-20453,19991005,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,473.24,NaN,NaN,NaN
10,3220000,3220000-101-1899-20463,19991006,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,282.59,NaN,NaN,NaN
15,3220000,3220000-101-1899-20500,19991012,NaN,1,영업/정상,1,영업,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,N,1157.90,NaN,NaN,NaN
30,3220000,3220000-101-1911-16480,19110528,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,90.75,NaN,NaN,NaN
55,3220000,3220000-101-1976-17313,19760724,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,39.26,NaN,NaN,NaN
56,3220000,3220000-101-1976-17996,19761223,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,50.24,NaN,NaN,NaN
83,3220000,3220000-101-1978-12840,19781006,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,39.54,NaN,NaN,NaN
91,3220000,3220000-101-1978-16489,19781017,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,49.24,NaN,NaN,NaN


In [89]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)


In [84]:
# 지역+메뉴 이름 조합으로 url에 미리 포함시키는 driver.get 함수 , count는 검색했을때 조회하는 범위
def naver_finder(place,count) :

  # 검색어를 네이버 url에 포함시켜 이동 + 검색 
  driver.get('https://map.naver.com/p/search/{}'.format(place))

  # 저장 
  naver_df = pd.DataFrame()
  time.sleep(3)  
  try : 
    # 목록의 1~9번째까지 반복
    for num in range(1,count+1) :
      # frame을 3갤 나눔 searchIframe(왼쪽), entryIframe(오른쪽),default_content(기본)

      # 왼쪽 프레임
      searchIframe = driver.find_element(By.ID,'searchIframe')
      driver.switch_to.frame(searchIframe)

      try : 
        # {}안에 num 입력 (1번째, 2번째...)
        # 각 카드 상단을 클릭하여 WebDriverWait을 활용 카드 상단 XPATH가 보일때까지 3초 대기 
        driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num)).click()

      except :
        # 가능하지 않다면 스크롤 내리고 최대 3초까지 기다렸다가 클릭
        # tag 내용이 안 보이면 해당 tag에서 스크롤 내림
        # click()이 안되는 경우 클릭을 대체하여 사용하기 
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
        card_clik = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
        driver.execute_script('arguments[0].click();',card_clik)

      # 기본 프레임
      time.sleep(1)
      driver.switch_to.default_content()

      # 오른쪽 프레임
      entryIframe = driver.find_element(By.ID,'entryIframe')
      driver.switch_to.frame(entryIframe)

      # 가게 이름, 변수로 지정
      restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text

      # 가게 별점이 있는 경우에만 try 
      try :
        review_star = driver.find_element(By.CLASS_NAME,'PXMot').text
        review_star = re.sub('방문자리뷰',"",review_star)
      except :
        review_star = 0
        pass 

      # 기본 설정으로 돌아오기
      time.sleep(1)
      driver.switch_to.default_content()

      print(restaurant_name,review_star)
      
      # 수정 필요 
      df = pd.DataFrame({'가게명':[restaurant_name],'네이버별점' : [review_star]})
      naver_df = pd.concat([naver_df,df])
      naver_df.drop_duplicates() # 중복 제거 
  except :
    print('정상적으로 작동이 되지 않습니다.')

    return naver_df


In [92]:
for i in df_top10['사업장명']:
    df_sample = naver_finder(i,2)
    time.sleep(5)

정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
하늘사다리 별점
4.43
은희네 온집닭떡볶이 상도본점 별점
4.47
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.


""


""
